# serde.dataclass

<font size = 3>Serde.dataclass is a dataclass decorator which is based on Pydantic dataclass. It provides value check which is similar with Pydantic but it also provides a way for us to save class.</font>

## How to use it

In [1]:
from pydantic import Field
from gluonts.core import serde

<font size=3>For example, it can check value types and make automatically conversion. You can add more logic after the type checking and conversion in `__post_init_post_parse` method.</font>

In [2]:
@serde.dataclass
class A:
    v1: int
    v2: float
    
    def __post_init_post_parse__(self):
        self.v3 = self.v1 + self.v2

In [3]:
a = A(1.3, 2)
print(a.v1, a.v2, a.v3)
print(repr(a))

1 2.0 3.0
__main__.A(v1=1.3, v2=2)


<font size=3>You can also check the range by using Field. For example, if you want v1 be greater than 0, you can use `v1: int = Field(gt=0)`. However, because you give v1 a restriction, you should make v2 a `Field(...)`. Pydantic would help you check if you input v2.</font>

In [4]:
@serde.dataclass
class B:
    v1: int = Field(gt=0)
    v2: float = Field(...)
    
    def __post_init_post_parse__(self):
        self.v3 = self.v1 + self.v2

In [5]:
try:
    b1 = B(-1, 2)   
except Exception as e:
    print("b1 error:", e)
try:
    b2 = B(2)   
except Exception as e:
    print("b2 error:", e)
try:
    b3 = B(1.5, 3)   
except Exception as e:
    print("b3 error:", e)

b1 error: 1 validation error for B
v1
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)
b2 error: 1 validation error for B
v2
  field required (type=value_error.missing)


<font size=3>serde.dataclass also performs well in inheritance. However, sometimes you should also use Field(...) because there are some fields with default value in the base class. You can only use args to set the value in subclass in order. For other values you should use kwargs. Besides, if you use `__post_init_post_parse__()` don't forget to check if the base class also use it.</font>

In [6]:
from typing import List
@serde.dataclass
class C(B):
    v2: float = Field(...)
    v4: List[int] = Field(gt=0)
    
    def __post_init_post_parse__(self):
        super().__post_init_post_parse__()
        self.v5 = "dataclass"

In [7]:
try:
    c1 = C(3, [1,3])
    c1.v1, c1.v2, c1.v3, c1.v4, c1.v5
except Exception as e:
    print("c1 error:", e)
try:
    c2 = C(3.6, [-1,3], v1=3)
    c2.v1, c2.v2, c2.v3, c2.v4, c2.v5
except Exception as e:
    print("c2 error:", e)
try:
    c3 = C(3.6, [1,3], v1=3)
    c3.v1, c3.v2, c3.v3, c3.v4, c3.v5
except Exception as e:
    print("c3 error:", e)

c1 error: 1 validation error for C
v1
  field required (type=value_error.missing)
c2 error: 1 validation error for C
v4 -> 0
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)


<font size=3>Besides, you can use OrElse, A default field for a dataclass, that uses a function to calculate thevalue if none was passed.The function can take arguments, which are other fields of the annotateddataclass, which can also be `OrElse` fields. In case of circular dependencies an error is thrown.</font>

In [8]:
@serde.dataclass
class D:
    v1: int
    v2: int = Field(serde.OrElse(lambda v1: v1 * 2))
    v3: int = Field(serde.OrElse(lambda v2: v2 + 4))

In [9]:
d1 = D(10)
print(d1.v1, d1.v2, d1.v3)
d2 = D(10, 17)
print(d2.v1, d2.v2, d2.v3)

10 20 24
10 17 21


<font size=3>You can also add some additionl kwargs, dataclass would help you make them `self.kwargs`</font>.

In [10]:
@serde.dataclass
class E:
    v1: int
    def __post_init_post_parse__(self):
        self.a = 0
        for key, value in self.kwargs.items():
            self.a += value

In [11]:
e = E(v1=1, a1=2, a2=5, a3=8)
print(e.v1, e.a)

1 15


<font size=3>If the field value may change when you use it, please add the member in `__init_kwargs__`.</font>

In [12]:
@serde.dataclass
class F:
    v1: List[int]
    v2: float = 3.0

    def add_v1(self):
        self.v1[0] += 10
        
f1 = F([0])
f1.add_v1()
f2 = serde.decode(serde.encode(f1))
print("f1.v1:", f1.v1)
print("f2.v1:", f2.v1)

f1.v1: [10]
f2.v1: [0]


In [13]:
@serde.dataclass
class F:
    v1: List[int]
    v2: float = 3.0

    def __post_init_post_parse__(self):
        self.__init_kwargs__["v1"] = self.v1

    def add_v1(self):
        self.v1[0] += 10
        
f1 = F([0])
f1.add_v1()
f2 = serde.decode(serde.encode(f1))
print("f1.v1:", f1.v1)
print("f2.v1:", f2.v1)

f1.v1: [10]
f2.v1: [10]


## Remain problems

### 1. Fix and use EVENTUAL

### 2. OrElse

<font size=3>It not works well if the field is assigned in `__post_init_post_parse__`</font>

In [14]:
@serde.dataclass
class G:
    v1: List[int] = None
    v2: List[int] = Field(serde.OrElse(lambda v1: [a * 2 for a in v1]))
    def __post_init_post_parse__(self):
        if self.v1 is None:
            self.v1 = [1]

In [15]:
try:
    g1 = G([7])
    print(g1.v1, g1.v2)
except Exception as e:
    print("g1 error:", e)
try:
    g2 = G()
    print(g2.v1, g2.v2)
except Exception as e:
    print("g2 error:", e)

[7] [14]
g2 error: 'v1'


### 3. Checkings in `__post_init_post_parse__()`

<font size=3>For example in `src/gluonts/mx/model/seq2seq/_mq_dnn_estimator.py`. In this case we put the `__post_init_post_parse__` method of the base class at that of the end of the subclass.</font>

In [16]:
@serde.dataclass
class ForkingSeq2SeqEstimator():
    distr_output: int = Field(None)
    quantile_output: float = Field(None)

    def __post_init_post_parse__(self):
        assert (self.distr_output is None) != (self.quantile_output is None)

In [17]:
@serde.dataclass
class MQCNNEstimator1(ForkingSeq2SeqEstimator):
    distr_output: int = Field(None)
    quantile_output: float = Field(None)

    def __post_init_post_parse__(self):
        self.distr_output = 1
        super().__post_init_post_parse__()

try:
    MQCNNEstimator1()
except Exception as e:
    print(e)

In [18]:
@serde.dataclass
class MQCNNEstimator2(ForkingSeq2SeqEstimator):
    distr_output: int = Field(None)
    quantile_output: float = Field(None)

    def __post_init_post_parse__(self):
        super().__post_init_post_parse__()
        self.distr_output = 1
        
try:
    MQCNNEstimator2()
except Exception as e:
    print("assert error")

assert error


<font size=3>But in this case, we should put it at the begining.</font>

In [19]:
@serde.dataclass
class Baseclass:
    v1: int
    v2: int
    def __post_init_post_parse__(self):
        self.v3 = self.v1 + self.v2

In [20]:
@serde.dataclass
class Subclass(Baseclass):
    v4: int
    def __post_init_post_parse__(self):
        super().__post_init_post_parse__()
        self.v5 = self.v3 * self.v4

In [21]:
s = Subclass(v1=2, v2=5, v4=7)
s.v3, s.v5

(7, 49)

<font size=3>I think we may not use dataclass on abstract class.</font>